Copyright 2024 Google, LLC. This software is provided as-is,
without warranty or representation for any use or purpose. Your
use of it is subject to your agreement with Google.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

   http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

# Example Agent Workflow using Google's ADK

This notebook provides an example of building an agentic workflow with Google's new ADK. For more information please visit https://google.github.io/adk-docs/

# Multi-Agent Nest Support System with Vertex AI RAG and ADK

This notebook demonstrates building a customer support agent system using Vertex AI's Generative Models, RAG (Retrieval-Augmented Generation) Engine, and the Agent Development Kit (ADK).

The system includes:
1.  **RAG Setup:** Ingesting Nest support documents into a Vertex AI RAG Corpus.
2.  **Tool Definitions:** Functions to interact with a mock ticketing system and manage files.
3.  **Sub-Agent Definitions:** Specialized agents for RAG retrieval, reasoning/process definition, and adding notes to tickets.
4.  **Root Agent Definition:** An orchestrator agent that delegates tasks to the sub-agents.
5.  **Interaction:** Running a sample conversation with the agent team.

## 0. Install Dependencies

Install the necessary libraries for Vertex AI, Google Cloud, ADK, and HTTP requests.

In [ ]:
# Core Vertex AI, RAG and Agent SDK
!pip install --upgrade --quiet google-cloud-aiplatform google-cloud-storage requests google-cloud-discoveryengine
!pip install --upgrade --quiet google-adk

# Optional: For rendering markdown in output
!pip install --upgrade --quiet IPython

print("Dependencies installed/updated.")

## 1. Import Libraries

In [1]:
# Vertex AI Modules
import vertexai
from vertexai.preview.generative_models import GenerativeModel, GenerationConfig, Part, Tool, ChatSession, FunctionDeclaration, grounding, GenerationResponse
from vertexai.preview import rag

# Vertex Agent Modules
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.artifacts.in_memory_artifact_service import InMemoryArtifactService
from google.adk.tools.agent_tool import AgentTool


# Vertex GenAI Modules
import google.genai
from google.genai import types

# Google Cloud AI Platform Modules
from google.cloud import aiplatform_v1beta1 as aiplatform # This module helps parse the info for delete_rag_corpa function
from google.cloud import storage

#Google BigQuery
from google.cloud import bigquery


# Other Python Modules
#import base64
#from IPython.display import Markdown
import asyncio
import requests
import os
from typing import List, Dict, TypedDict, Any
import json
from urllib.parse import urlparse
import warnings
import logging

print("Libraries imported successfully.")

Libraries imported successfully.


Ignore warning messages

In [2]:
warnings.filterwarnings("ignore")
logging.basicConfig(level=logging.ERROR)

## 2. Configuration

**Important:** Update the `project_id`, `corpa_document_bucket`, `local_documents`, and `ticket_server_url` variables below with your specific values.

In [3]:
project_id = "YOUR_PROJECT_ID" # Your GCP Project ID
location = "global" # You can leave this setting as global
region = "us-central1" # Your region. This notebook has only been tested in us-central1

corpa_name = "nest-rag-corpus" # This will be the display name of your RAG Engine corpus

corpa_document_bucket = "gs://YOUR_BUCKET_ID/nest/docs/" # The GCS path to the files you want to ingest into your RAG Engine corpus
bq_data_bucket = "gs://YOUR_BUCKET_ID/wip/bq_import/" # The GCS path to the files you want to ingest into your BQ datastore

support_documents = "./nest_docs/" # Local directory containing Nest support files to copy
bq_data = "./bq_data/" # Local directory containing BQ data files

ticket_server_url = "http://ticket01:8000" # The url to the mock ticket system. This will be a GCE VM running the ticket_server.py web service.

## 3. Environment Setup and Vertex AI Initialization

Set environment variables for Google libraries and initiate the vertex ai client

In [4]:
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "1"
os.environ["GOOGLE_CLOUD_PROJECT"] = project_id
os.environ["GOOGLE_CLOUD_LOCATION"] = region

In [5]:
vertexai.init(project=project_id, location=region)

## 4. Google Cloud Storage Setup

This function checks if the specified GCS bucket and folder exist, creates them if necessary, and uploads documents from the local directory.

In [6]:
def create_bucket(bucket_path: str):
    parsed_uri = urlparse(bucket_path)
    bucket_name = parsed_uri.netloc
    prefix = parsed_uri.path.lstrip('/')

    storage_client = storage.Client()

    # Get the bucket object
    bucket = storage_client.bucket(bucket_name)

    # Check if the bucket exists, create it if not
    if not bucket.exists():
        bucket.create()
        print(f"Bucket '{bucket_name}' created successfully.")
    else:
        print(f"Bucket '{bucket_name}' already exists.")

    # Create the folder prefix if it doesn't implicitly exist
    if prefix:
        blob_name = f"{prefix}" if prefix.endswith('/') else f"{prefix}/"
        placeholder_blob = bucket.blob(blob_name + ".placeholder")
        if not placeholder_blob.exists():
            placeholder_blob.upload_from_string('')
            print(f"Simulated folder '{bucket_path}' created.")
        else:
            print(f"Simulated folder '{bucket_path}' already exists.")

In [11]:
def upload_files(folder_path: str, bucket_path: str):
    parsed_uri = urlparse(bucket_path)
    bucket_name = parsed_uri.netloc
    prefix = parsed_uri.path.lstrip('/')
    
    storage_client = storage.Client()

    # Get the bucket object
    bucket = storage_client.bucket(bucket_name)

    if os.path.exists(folder_path) and os.path.isdir(folder_path):
        for filename in os.listdir(folder_path):
            local_file_path = os.path.join(folder_path, filename)
            if os.path.isfile(local_file_path):
                gcs_blob_name = f"{prefix}{filename}"
                blob = bucket.blob(gcs_blob_name)
                blob.upload_from_filename(local_file_path)
                print(f"Uploaded '{local_file_path}' to 'gs://{bucket_name}/{gcs_blob_name}'")
    else:
        print(f"Local directory '{folder_path}' does not exist or is not a directory.")

Create the Nest Support Document bucket

In [8]:
create_bucket(corpa_document_bucket)

Bucket 'YOUR_BUCKET_ID' already exists.
Simulated folder 'gs://YOUR_BUCKET_ID/nest/docs/' already exists.


Create the BQ Data bucket

In [9]:
create_bucket(bq_data_bucket)

Bucket 'YOUR_BUCKET_ID' already exists.
Simulated folder 'gs://YOUR_BUCKET_ID/wip/bq_import/' created.


Upload the Nest support documents to the GCS bucket

In [12]:
upload_files(support_documents, corpa_document_bucket)

Uploaded './nest_docs/Nest_Power_Connector_Installation_Guide.pdf' to 'gs://YOUR_BUCKET_ID/nest/docs/Nest_Power_Connector_Installation_Guide.pdf'
Uploaded './nest_docs/nest-thermostat-gen3-install-guide-US.pdf' to 'gs://YOUR_BUCKET_ID/nest/docs/nest-thermostat-gen3-install-guide-US.pdf'
Uploaded './nest_docs/nest-thermostat-e-install-guide.pdf' to 'gs://YOUR_BUCKET_ID/nest/docs/nest-thermostat-e-install-guide.pdf'


Upload the BQ data documents to the GCS bucket

In [13]:
upload_files(bq_data, bq_data_bucket)

Uploaded './bq_data/product_data.avro' to 'gs://YOUR_BUCKET_ID/wip/bq_import/product_data.avro'


## 5. Helper Functions

Define functions for interacting with the agent, managing RAG corpora, and defining tools.

In [28]:
# @title Define Agent Interaction Function
async def call_agent_async(query: str, runner: Runner, user_id: str, session_id: str):
    """Sends a query to the agent runner and prints the final response."""
    #print(f"\n>>> User Query: {query}")

    # Prepare the user's message in ADK format
    content = types.Content(role='user', parts=[types.Part(text=query)])

    final_response_text = "Agent did not produce a final response." # Default

    try:
        # Key Concept: run_async executes the agent logic and yields Events.
        # We iterate through events to find the final answer.
        async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
            # You can uncomment the line below to see *all* events during execution
            # print(f"  [Event] Author: {event.author}, Type: {type(event).__name__}, Final: {event.is_final_response()}, Content: {event.content}")

            # Key Concept: is_final_response() marks the concluding message for the turn.
            if event.is_final_response():
                if event.content and event.content.parts:
                    # Assuming text response in the first part
                    final_response_text = event.content.parts[0].text
                elif event.actions and event.actions.escalate: # Handle potential errors/escalations
                    final_response_text = f"Agent escalated: {event.error_message or 'No specific message.'}"
                # Add more checks here if needed (e.g., specific error codes)
                break # Stop processing events once the final response is found

    except Exception as e:
        print(f"Error during agent execution: {e}")
        final_response_text = f"Agent interaction failed with error: {e}"

    print(f"Agent: {final_response_text}") # Added "Agent: " prefix

print("Agent interaction function `call_agent_async` defined.")

Agent interaction function `call_agent_async` defined.


In [29]:
# @title Define RAG Corpus Management Functions

# NOTE: The delete function is defined but not used in the main flow.
# It can be useful for cleanup. It expects a pager object, which is
# typically obtained from `rag.list_corpora()`.
def delete_rag_corpora(rag_corpora_pager: aiplatform.services.vertex_rag_data_service.pagers.ListRagCorporaPager):
    """
    Deletes all RAG corpora listed in the provided pager object.
    USE WITH CAUTION! THIS WILL PERMANENTLY DELETE CORPORA.

    Args:
        rag_corpora_pager: The pager object from rag.list_corpora().
    """
    names_list = []
    print("Identifying corpora to delete...")
    try:
        for rag_corpus_obj in rag_corpora_pager: # Iterate through the actual RagCorpus objects
            if hasattr(rag_corpus_obj, 'name'):
                print(f" - Found corpus: {rag_corpus_obj.display_name} ({rag_corpus_obj.name})")
                names_list.append(rag_corpus_obj.name)
            else:
                print(f" - Skipping object without a 'name' attribute: {rag_corpus_obj}")
    except Exception as e:
        print(f"Error iterating through corpora pager: {e}")
        return # Stop if we can't list them properly

    if not names_list:
        print("No corpora found to delete.")
        return

    print("\nStarting deletion process...")
    deleted_count = 0
    failed_count = 0
    for corpus_name_to_delete in names_list:
        print(f"Attempting to delete corpus: {corpus_name_to_delete}...")
        try:
            # Optional: You might want to double-check existence before deleting
            # rag.get_corpus(name=corpus_name_to_delete)
            rag.delete_corpus(name=corpus_name_to_delete, force=True) # Use force=True to delete non-empty corpora
            print(f"  Successfully deleted {corpus_name_to_delete}")
            deleted_count += 1
        except Exception as e:
            print(f"  Failed to delete {corpus_name_to_delete}: {e}")
            failed_count += 1
    print(f"\nDeletion complete. Deleted: {deleted_count}, Failed: {failed_count}")


def create_rag_corpora(display_name, source_bucket):
    EMBEDDING_MODEL = "publishers/google/models/text-embedding-004"  # @param {type:"string", isTemplate: true}
    embedding_model_config = rag.EmbeddingModelConfig(publisher_model=EMBEDDING_MODEL)

    rag_corpus = rag.create_corpus(
        display_name=display_name, embedding_model_config=embedding_model_config
    )
    

    
    INPUT_GCS_BUCKET = (
        source_bucket
    )

    response = rag.import_files(
        corpus_name=rag_corpus.name,
        paths=[INPUT_GCS_BUCKET],
        chunk_size=1024,  # Optional
        chunk_overlap=100,  # Optional
        max_embedding_requests_per_min=900,  # Optional
    )
    
    # This code shows how to upload local files to the corpus. 
    #rag_file = rag.upload_file(
    #    corpus_name=rag_corpus.name,
    #    path="./test.txt",
    #    display_name="test.txt",
    #    description="my test file"
    #)
    
    return rag_corpus

    

print("RAG corpus management functions defined.")

RAG corpus management functions defined.


In [30]:
# @title Define Tool Functions (Ticketing, File Handling, RAG Query)

# --- Ticketing Tools ---

def create_ticket(ticket_data: Dict) -> Dict:
    """
    Creates a ticket via the /ticket endpoint.

    Args:
        ticket_data: A dictionary containing the ticket data
                     (ticket_id, description, customer_id, contact_name).

    Returns:
        A dictionary containing the API response.  Handles errors gracefully.
    """
    url = f"{ticket_server_url}/ticket"
    headers = {"Content-Type": "application/json"}

    try:
        response = requests.post(url, headers=headers, data=json.dumps(ticket_data))
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
        return response.json()
    except requests.exceptions.RequestException as e:
        return {"error": f"Request failed: {e}"}

In [31]:
def add_note(ticket_id: int, contact_name: str, note: str) -> Dict[str, Any]:
    """
    Adds a note to a ticket via the API's /notes endpoint.

    Sends the provided ticket details as JSON to the API and returns
    the parsed JSON response from the server.

    Args:
        ticket_id: int - The ID number of the ticket.
        contact_name: str - The name of the contact person.
        note: str - The content of the note to add.

    Returns:
         Dict[str, Any]:
            - On success: A dictionary representing the parsed JSON response
              from the API (content depends on the specific API implementation,
              often includes details of the created note or a success status).
            - On failure (request exception or non-2xx HTTP status):
              A dictionary containing an 'error' key with a description of the failure,
              e.g., {"error": "Request failed: 404 Client Error: Not Found for url: ..."}.
    """
    url = f"{ticket_server_url}/notes"
    headers = {"Content-Type": "application/json"}

    # Construct the dictionary *inside* the function
    note_data = {
        "ticket_id": ticket_id,
        "contact_name": contact_name,
        "note": note
    }

    try:
        response = requests.post(url, headers=headers, data=json.dumps(note_data))
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        return {"error": f"Request failed: {e}"}

In [32]:
def add_file_to_session(uri: str) -> str:
    """
    Adds a specific file from Google Cloud Storage (GCS) to the current session state for agent processing.

    This function takes a GCS URI for a file and wraps it in a `types.Content` object.
    This object is then typically used to make the file's content accessible to the
    agent for tasks like summarization, question answering, or data extraction
    related specifically to that file within the ongoing conversation or session.
    The MIME type is assumed to be inferred by the underlying system or defaults.

    Use this function *after* you have identified a specific GCS URI (e.g., using
    `get_gcs_uri` or similar) that you need the agent to analyze or reference directly.

    Args:
        uri: str - The complete Google Cloud Storage URI of the file to add.
                 Must be in the format "gs://bucket_name/path/to/file.pdf".
                 Example: "gs://my-doc-bucket/reports/q1_report.pdf"

    Returns:
         types.Content - A structured Content object representing the referenced file.
                       This object has `role='user'` and contains a `types.Part`
                       that holds the reference to the provided GCS URI.
                       This Content object can be passed to the agent in subsequent calls.
    """
    print(uri)
    content = types.Content(
        role='user',
        parts=[
            # Try passing ONLY the uri positionally, based on the error message "takes 1 positional argument"
            types.Part.from_uri(
                file_uri=uri,
                mime_type="application/pdf",
            )
        ]
    )

    return content

In [33]:
def get_gcs_uri(query: str) -> str:
    """
    Retrieves Google Cloud Storage (GCS) URIs for documents relevant to a given query.

    This function queries a pre-configured Retrieval-Augmented Generation (RAG)
    corpus to find documents related to the input query string. It extracts
    the source GCS URIs from the top relevant documents identified by the
    RAG system based on semantic similarity. Use this function when you need
    to find the source files in GCS that contain information related to a
    specific question or topic.

    Args:
        query: str - The natural language query or topic to search for within
                 the RAG corpus. For example: "What were the Q3 sales figures?"
                 or "Tell me about project Alpha's latest status".

    Returns:
         str - A JSON string representing a list of unique GCS URIs. These URIs
               point to the source documents found to be relevant to the query.
               Returns a JSON string representing an empty list ('[]') if no
               relevant documents meet the similarity criteria.
               Example return value: '["gs://my-bucket/doc1.pdf", "gs://my-bucket/report_q3.txt"]'
    """
    query_response = rag.retrieval_query(
        rag_resources=[
            rag.RagResource(
                rag_corpus=rag_corpus.name,
                # Optional: supply IDs from `rag.list_files()`.
                # rag_file_ids=["rag-file-1", "rag-file-2", ...],
            )
        ],
        text=f'''
        {query}
        ''',
        similarity_top_k=10,  # Optional
        vector_distance_threshold=0.5,  # Optional
    )
    #print(response)
    uri_set = set()
    for context in query_response.contexts.contexts:
        uri_set.add(context.source_uri)
        #json.dumps(list(uri_set))
    #doc_uri = uri_set.pop()
    doc_uri = json.dumps(list(uri_set))
    return doc_uri

## 6. RAG Corpus Setup

Check if the RAG Corpus configured in step 2 exists. If not, create it and initiate file import.

In [34]:
existing_corpora = rag.list_corpora()

print(existing_corpora)

# Variable to hold the corpus if found
found_corpus = None

ListRagCorporaPager<rag_corpora {
  name: "projects/YOUR_PROJECT_ID/locations/us-central1/ragCorpora/6512768011131158528"
  display_name: "nest-rag-corpus"
  create_time {
    seconds: 1745333337
    nanos: 950261000
  }
  update_time {
    seconds: 1745333337
    nanos: 950261000
  }
  rag_embedding_model_config {
    vertex_prediction_endpoint {
      endpoint: "projects/YOUR_PROJECT_ID/locations/us-central1/publishers/google/models/text-embedding-004"
    }
  }
  rag_vector_db_config {
    rag_managed_db {
    }
    rag_embedding_model_config {
      vertex_prediction_endpoint {
        endpoint: "projects/YOUR_PROJECT_ID/locations/us-central1/publishers/google/models/text-embedding-004"
      }
    }
  }
  corpus_status {
    state: ACTIVE
  }
  vector_db_config {
    rag_managed_db {
    }
    rag_embedding_model_config {
      vertex_prediction_endpoint {
        endpoint: "projects/YOUR_PROJECT_ID/locations/us-central1/publishers/google/models/text-embedding-004"
      }
    }
 

In [35]:
# Iterate through all existing RAG corpora
for corpus in existing_corpora.rag_corpora: # Ensure you iterate the correct attribute
    # Check if display_name exists and matches
    if getattr(corpus, 'display_name', None) == corpa_name:
        print(f"Existing Corpa found. Using {corpus.name}")
        
        # You already have the corpus object, no need to call get_corpus usually
        # If 'corpus' object from the list is sufficient, use it directly.
        # If you MUST get a fresh object or different type, uncomment the next line:
        # rag_corpus = rag.get_corpus(name=corpus.name) 
        found_corpus = corpus # Store the found corpus object
        
        print(f"This corpus contains the following files:")
        try:
            # List files associated with the found corpus
            for file in rag.list_files(corpus.name): # Use corpus.name
                print(getattr(file, 'display_name', 'N/A')) # Safer access
        except Exception as e:
            print(f"Warning: Could not list files for {corpus.name}. Error: {e}")
            
        break # Exit the loop as soon as we find the match

# After the loop, check if we found anything
if found_corpus is None:
    # The loop completed without finding the corpus
    print(f"No existing {corpa_name} resource found. Creating one now.")
    try:
        rag_corpus = create_rag_corpora(corpa_name, corpa_document_bucket)
        print(f"New RAG corpus created at {rag_corpus.name}")
    except Exception as e:
        print(f"Error creating corpus {corpa_name}: {e}")
        rag_corpus = None # Indicate failure
else:
    # The corpus was found in the loop
    rag_corpus = found_corpus # Assign the found corpus to the main variable

# Now 'rag_corpus' holds either the found or newly created corpus (or None if creation failed)
# You can proceed to use 'rag_corpus' here
if rag_corpus:
    print(f"\nProceeding with corpus: {rag_corpus.name}")
    # ... your next steps using rag_corpus ...
else:
    print(f"\nFailed to find or create corpus '{corpa_name}'. Cannot proceed.")

Existing Corpa found. Using projects/YOUR_PROJECT_ID/locations/us-central1/ragCorpora/6512768011131158528
This corpus contains the following files:
nest-thermostat-e-install-guide.pdf
Nest_Power_Connector_Installation_Guide.pdf
nest-thermostat-gen3-install-guide-US.pdf

Proceeding with corpus: projects/YOUR_PROJECT_ID/locations/us-central1/ragCorpora/6512768011131158528


In [ ]:
test = get_gcs_uri('How do I install a Nest E thermostat')
print(test)

## 8. Define Sub-Agents

Define the specialized agents:
1.  `rag_agent`: Finds relevant documents using the RAG tool.
2.  `reasoning_agent`: Outlines troubleshooting steps based on provided documents.
3.  `notes_agent`: Adds notes to the ticketing system.

In [ ]:
# --- RAG Agent ---
rag_agent = None
try:
    rag_agent = Agent(
        model="gemini-2.0-flash-001",
        name="rag_agent",
        instruction=
        """
          You are a customer support agent. You help locate documentation that will resolve customer issues.
          Identify the most relevant support document that pertains to the question.
          Your job is to only provide the GCS URI for the closest matching document, not to resolve the issue.
          You will use the get_gcs_uri to identify the correct file. 
          The response from get_gcs_uri will be a text string like 'gs://bucket_name/folder/file 1.pdf'
          Determine which files are relevant to the customer's question and return them to the root agent.
        """,
        description="Retrieves information from a RAG Engine instance and returns the GCS URI of relevant files.",
        tools=[
            get_gcs_uri
        ],
    )
    print(f"✅ Agent '{rag_agent.name}' created.")
except Exception as e:
    print(f"❌ Could not create Greeting agent. Error: {e}")

In [ ]:
# -- Reasoning Agent ---
reasoning_agent = None
try:
    reasoning_agent = Agent(
        #model="gemini-2.5-pro-exp-03-25", # Using a different model to account for potential resource constraints
        model="gemini-2.5-flash-preview-04-17",
        name="reasoning_agent",
        instruction=
        """
          You are a customer support agent. You help define the troubleshooting process to resolve customer issues.
          Use the information in the document to define the process for resolving the issue.
          Once the files have been added to your context, use that information to outline the process needed to resolve the identified problem.
          If multiple documents are provided, specify which document or documents contains the relevant information to resolve the issue.
          The process needs to outline the activities for the Nest technical support representitive to perform.
          The notes system only supports plain text. Ensure you only use text in your output.
      
          Example:
          Step 1: Do this
          Step 2: Do this other task
          Step 3: etc
        """,
        description="Defines the troubleshooting process to help resolve the customer's problem",
        tools=[
            #add_file_to_session,
        ],
    )
    print(f"✅ Agent '{reasoning_agent.name}' created.")
except Exception as e:
    print(f"❌ Could not create Greeting agent. Error: {e}")

In [ ]:
# --- Notest Agent ---
notes_agent = None
try:
    notes_agent = Agent(
        model="gemini-2.0-flash-001",
        name="notes_agent",
        instruction="""
            You are a customer support assistant agent. Your primary task is to generate informative and well-structured notes for customer support tickets. 
            These notes will be added to the ticket's history and used by customer support representatives to understand the issue, track progress, and provide consistent service.
            Step 1: Check if you have the user's ticket ID and contact name (you can use their provided name or email as contact name). If not, politely ask for the informaiton you are missing.
            Step 2: Once you have the ticket ID, contact name, and the troubleshooting steps, use the 'add_note' tool to add the notes to the ticket.
            Step 3: Confirm to the user that the ticket has been updated with the troubleshooting plan.
            Step 4: Thank the customer for their time and to have a nice day.
        """,
        description="Add notes to the associated ticket",
        tools=[
            add_note,
        ],
    )
    print(f"✅ Agent '{notes_agent.name}' created.")
except Exception as e:
    print(f"❌ Could not create Greeting agent. Error: {e}")

## 9. Define Root Agent

Define the main orchestrator agent (`nest_agent_team`) that coordinates the sub-agents and uses the `add_file_to_session` tool.

In [ ]:
# @title Define the Root Agent with Sub-Agents

# Ensure sub-agents were created successfully before defining the root agent.
# Also ensure the original 'get_weather' tool is defined.
root_agent = None
runner_root = None # Initialize runner

if rag_agent and reasoning_agent and notes_agent and 'add_file_to_session' in globals():
    # Let's use a capable Gemini model for the root agent to handle orchestration
    root_agent_model = 'gemini-2.0-flash-001'
        
    nest_agent_team = Agent(
        name="nest_support_agent",
        model="gemini-2.0-flash-001",
        description="The main coordinator agent. Handles user requests and delegates tasks to specialists.",
        instruction=
        """
            You are the lead Nest customer support coordinator agent. Your goal is to understand the customer's issue, find relevant documentation, generate a troubleshooting plan, and log the plan into their support ticket.

            You have access to specialized tools and sub-agents:
            1. Tool `add_file_to_session`: Use this tool *after* getting GCS URIs. Provide ONE GCS URI (e.g., "gs://bucket/doc.pdf") to this tool. The tool prepares the file content for context. Call this tool for EACH relevant URI returned by the rag_agent.
            2. Sub-Agent `rag_agent`: Call this agent first with the user's problem description to get a JSON list of relevant GCS document URIs.
            3. Sub-Agent `reasoning_agent`: Call this agent *after* using `add_file_to_session` for all relevant URIs. Provide the user's problem and indicate that the relevant documents are now in context. This agent will return the troubleshooting steps.
            4. Sub-Agent `notes_agent`: Call this agent last. You need the `ticket_id` (integer), `contact_name` (string, use your name "Nest Support Agent"), and the `note` (string, the troubleshooting steps from reasoning_agent). Ask the user for their ticket ID and name/email if you don't have it.

            Follow these steps precisely:
            Step 1: Greet the user and understand their Nest product issue.
            Step 2: Respond to the customer with an acknowledgement of the question and politely ask the customer to please wait while you research the answer.
            Step 3: Call the `rag_agent` with the user's issue description. Extract the GCS URIs from the JSON list it returns.
            Step 4: If URIs are found:
                - For EACH URI in the list, call the `add_file_to_session` tool with that single URI. Acknowledge to yourself or briefly mention you're processing the file.
            Step 5: After processing all relevant files, call the `reasoning_agent`. Provide the user's original problem description and explicitly state that the necessary documents are in the context. Capture the troubleshooting steps it returns.
            Step 6: Acknowledge to the user that you have the troubleshooting steps and provide a short summary to the customer.
            """,
        tools=[
            add_file_to_session,
            AgentTool(agent=rag_agent), 
            AgentTool(agent=reasoning_agent)],
        sub_agents=[notes_agent]
    )
    print(f"✅ Root Agent '{nest_agent_team.name}' created using model '{root_agent_model}' with sub-agents: {[sa.name for sa in nest_agent_team.sub_agents]}")

else:
    print("❌ Cannot create root agent because one or more sub-agents failed to initialize or 'add_file_to_session' tool is missing.")

## 10. Interact with the Agent Team

Now, set up the runner and session management to interact with the defined agent team.

In [ ]:
# @title Run Conversation with the Agent Team

# Ensure the root agent (e.g., 'nest_agent_team' or 'root_agent' from the previous cell) is defined.
# Ensure the call_agent_async function is defined.

# Check if the root agent variable exists before defining the conversation function
root_agent_var_name = 'root_agent' # Default name from Step 3 guide
if 'nest_agent_team' in globals(): # Check if user used this name instead
    root_agent_var_name = 'nest_agent_team'
elif 'root_agent' not in globals():
    print("⚠️ Root agent ('root_agent' or 'nest_agent_team') not found. Cannot define run_team_conversation.")
    # Assign a dummy value to prevent NameError later if the code block runs anyway
    root_agent = None

if root_agent_var_name in globals() and globals()[root_agent_var_name]:
    async def run_team_conversation():
        print("\n--- Testing Agent Team Delegation ---")
        # InMemorySessionService is simple, non-persistent storage for this tutorial.
        session_service = InMemorySessionService()

        # Define constants for identifying the interaction context
        APP_NAME = "nets_support_agent_team"
        USER_ID = "user_1_agent_team"
        SESSION_ID = "session_001_agent_team" # Using a fixed ID for simplicity

        # Create the specific session where the conversation will happen
        session = session_service.create_session(
            app_name=APP_NAME,
            user_id=USER_ID,
            session_id=SESSION_ID
        )
        print(f"Session created: App='{APP_NAME}', User='{USER_ID}', Session='{SESSION_ID}'")

        # --- Get the actual root agent object ---
        # Use the determined variable name
        actual_root_agent = globals()[root_agent_var_name]

        # Create a runner specific to this agent team test
        runner = Runner(
            agent=actual_root_agent, # Use the root agent object
            app_name=APP_NAME,       # Use the specific app name
            session_service=session_service # Use the specific session service
            )
        # Corrected print statement to show the actual root agent's name
        print(f"Runner created for agent '{actual_root_agent.name}'.")

        # Always interact via the root agent's runner, passing the correct IDs
        await call_agent_async(query = "Hello there!", runner=runner, user_id=USER_ID, session_id=SESSION_ID)
        await call_agent_async(query = "I need to know how to setup my nest gen 3 unit.", runner=runner, user_id=USER_ID, session_id=SESSION_ID)
        await call_agent_async(query = "My name is John Doe and email is johnDoe@here.com. My ticket number is 132436.", runner=runner, user_id=USER_ID, session_id=SESSION_ID)
        await call_agent_async(query = "Great, thank you!", runner=runner, user_id=USER_ID, session_id=SESSION_ID)

    # Execute the conversation
    # Note: This may require API keys for the models used by root and sub-agents!
    #asyncio.run(run_team_conversation())
    print("Attempting execution using 'await' (default for notebooks)...")
    await run_team_conversation()
else:
    print("\n⚠️ Skipping agent team conversation as the root agent was not successfully defined in the previous step.")

## 11. Cleanup (Optional)

If you want to delete the RAG Corpus created during this session, uncomment and run the following cell. **Warning:** This permanently deletes the corpus and its indexed data.

In [ ]:
# @title Delete the RAG Corpus (USE WITH CAUTION!)

# Set this flag to True only if you are sure you want to delete the corpus
confirm_delete = False

if confirm_delete and 'rag_corpus_resource_name' in globals() and rag_corpus_resource_name:
    print(f"Attempting to delete RAG Corpus: {rag_corpus_resource_name} ({corpa_name})")
    try:
        # We need the pager object for the delete function as written, but it's simpler
        # to delete directly by name if we know it.
        # Let's delete directly using the resource name.
        rag.delete_corpus(name=rag_corpus_resource_name, force=True) # force=True deletes even if not empty
        print(f"Successfully deleted corpus: {rag_corpus_resource_name}")
        rag_corpus = None
        rag_corpus_resource_name = ""
    except Exception as e:
        print(f"Failed to delete corpus {rag_corpus_resource_name}: {e}")
elif not ('rag_corpus_resource_name' in globals() and rag_corpus_resource_name):
    print("Skipping deletion: RAG corpus resource name is not set.")
else:
    print("Skipping deletion: confirm_delete is set to False.")